# TITLE : MODELS OF DENSITY FROM COMPOSITIONAL DATA MWC7+ PRESSURE TEMP USING MACHINE LEARNING ALGORITHM

OBJECTIVE : TRINING OF MODELS FOR FUTURE SELECTION FOR PREDICTION OF DENSITY USING WIDE RANGE OF COMPOSITION DATA.

THIS FILE PREAPRED SEPRATED FROM TRAINING BECAUSE TO EDIT GRAPH AND DATA TABLE USED FOR PREDICTION NEED TO MANY TIME RE RUN THIS.

TO AVOID TRINING MULTIPAL TIMES THIS IS DONE IN SEPERATED FILES. HOWEVER REVIEWER CAN CROSS VERIFY SAME TRAINED MODELS USED FOR TESTING BY MATCHING HYPER PARAMETER OPTIMIZED DURING TRAINING USED HERE OR NOT. 

ALGORITHM APPLIED : LINEAR REGRESSION, SUPPORT VECTOR MACHINE, KNN, RANDOM FOREST, DECISION TREE, ANN , XGB

In [1]:
#DATA EXTRACTION, MANIPULATION, VIZULIZATION LIBRARY
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib as mpl

#STATISTICAL TOOLS LIBRARY
import scipy.stats as stat
import pylab 
from statsmodels.stats.outliers_influence import variance_inflation_factor

#DATA FETURES OPERATION LIBRARY
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler

#MODELING LIBRARY
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

#MODELLING OF DEEP LEARNING MODEL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch


#MODEL EVALUATION LIBRARY
from sklearn.metrics import mean_squared_error , mean_absolute_error, r2_score



#Model saving and loading files
import pickle
from keras.models import load_model


<ipython-input-1-c3f1ee80687e>:33: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
path = "C:\\STUDY DRIVE\\VISCOSITY PAPER THESIS\\BELOW BUBBLE POINT\\VISCOSITY COMPOSITION\\MODEL 1\\DATASOURCE\\"

train_file_name = "TRAIN.csv"
test_file_name = "TEST.csv"
train = pd.read_csv(path+train_file_name)
test = pd.read_csv(path+test_file_name)

model_path = "C:\\STUDY DRIVE\\VISCOSITY PAPER THESIS\\BELOW BUBBLE POINT\\VISCOSITY COMPOSITION\\MODEL 1\\MODELS\\"

export_data_path = "C:\\STUDY DRIVE\\VISCOSITY PAPER THESIS\\BELOW BUBBLE POINT\\VISCOSITY COMPOSITION\\MODEL 1\\EXPORTED DATA\\"

figure_path = "C:\\STUDY DRIVE\\VISCOSITY PAPER THESIS\\BELOW BUBBLE POINT\\VISCOSITY COMPOSITION\\MODEL 1\\FIGURES\\"


In [3]:
X_train = train.drop("VISCOSITY",axis = 1)
y_train = train.VISCOSITY
X_test = test.drop("VISCOSITY",axis = 1)
y_test = test.VISCOSITY

                              #### Load Models For testing purpose  ####

In [4]:
#All File Name variable
scaler_file = 'scaler.sav'
linear_file = 'linear_model.sav'
svr_file    = 'svr_model.sav'
dt_file     = 'dt_model.sav'
rf_file     = 'rf_model.sav'
knn_file    = 'knn_model.sav'
xgb_file    = 'xgb_model.sav'

ann_file    = 'ann_model.h5'

In [5]:
scaler            = pickle.load(open(model_path+scaler_file , "rb"))
linear_regression = pickle.load(open(model_path+linear_file , "rb"))
svr_regression    = pickle.load(open(model_path+svr_file , "rb"))
dt_regression     = pickle.load(open(model_path+dt_file , "rb"))
rf_regression     = pickle.load(open(model_path+rf_file , "rb"))
knn_regression    = pickle.load(open(model_path+knn_file , "rb"))
xgb_regression    = pickle.load(open(model_path+xgb_file , "rb"))

ann_regression    = load_model(model_path+ann_file)

                              #### First Scale All Inputs with scaler model  ####

In [6]:
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)

## 1. Linear Regression Model

                              #### Calculate VIF for features ####

In [7]:
VIF = [variance_inflation_factor(x_train , i) for i in range(0,x_train.shape[1])]  #shape is indicating number of columns which is argument for VIF

In [8]:
VIF_DataFrame = pd.DataFrame(VIF)
VIF_DataFrame = VIF_DataFrame.rename({0:"VIF"} , axis = 1)
VIF_DataFrame["FEATURES"] = X_train.columns
VIF_DataFrame = VIF_DataFrame[["FEATURES" , "VIF"]]
VIF_DataFrame

,FEATURES,VIF
0,H2S,5.417068
1,N2,9.401669
2,CO2,328.200536
3,C1,4309.523974
4,C2,395.684460
5,C3,829.874318
6,C4,355.192434
7,C5,89.743169
8,C6,83.480735
9,C7+,7879.757830


In [9]:
linear_regression.intercept_

0.9058744314720819

In [10]:
linear_regression.coef_

array([-0.04663508, -0.01772861, -0.5324967 , -1.92418905,  0.20442362,
       -1.61622318, -0.30825984, -0.59147327, -0.12543998, -2.15843549,
       -0.05074814, -0.31669304, -0.17464318])

In [11]:
linear_weight = pd.DataFrame()
linear_weight["Feature"] = X_train.columns
linear_weight["Weights"] = linear_regression.coef_
linear_weight

,Feature,Weights
0,H2S,-0.046635
1,N2,-0.017729
2,CO2,-0.532497
3,C1,-1.924189
4,C2,0.204424
5,C3,-1.616223
6,C4,-0.308260
7,C5,-0.591473
8,C6,-0.125440
9,C7+,-2.158435


In [12]:
linear_regression_test_Prediction = linear_regression.predict(x_test)
linear_regression_train_Prediction = linear_regression.predict(x_train)

In [13]:
linear_regression_trainR2 = linear_regression.score(x_train,y_train)
linear_regression_testR2 = linear_regression.score(x_test,y_test)
print(linear_regression_trainR2,linear_regression_testR2)

0.5575163641089571 0.5960919203258765


                     #### Model Result for train and test data for linear Regression ####

In [14]:
result_lr_train = pd.DataFrame()
result_lr_train["Viscosity (train)"] = y_train
result_lr_train["Predicted Viscosity (train)"] = linear_regression_train_Prediction
result_lr_train["Residues"] = abs(result_lr_train["Viscosity (train)"] - result_lr_train["Predicted Viscosity (train)"])
result_lr_train["Absolute Error (%)"] = result_lr_train["Residues"]*100/result_lr_train["Viscosity (train)"]

result_lr_test = pd.DataFrame()
result_lr_test["Viscosity (test)"] = y_test
result_lr_test["Predicted Viscosity (test)"] = linear_regression_test_Prediction
result_lr_test["Residues"] = abs(result_lr_test["Viscosity (test)"] - result_lr_test["Predicted Viscosity (test)"])
result_lr_test["Absolute Error (%)"] = result_lr_test["Residues"]*100/result_lr_test["Viscosity (test)"]




In [15]:
result_lr_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.905874,0.337471,57.085121
std,0.714770,0.533697,0.334058,111.940001
min,0.033511,-1.496117,0.012262,0.731081
25%,0.403509,0.574623,0.125450,20.265427
50%,0.625668,0.779901,0.230120,32.776772
75%,1.250000,1.287336,0.405909,58.605993
max,3.010000,2.468736,2.291510,1460.106085


In [16]:
result_lr_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.862746,0.271633,50.803625
std,0.593976,0.512350,0.260459,57.382915
min,0.027323,-0.036172,0.003375,0.713533
25%,0.367331,0.476375,0.103605,18.655458
50%,0.592982,0.696903,0.179636,30.614444
75%,1.106952,1.233146,0.357149,50.244287
max,2.435695,2.143179,1.353629,232.251061


In [115]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)

plt.rcParams["font.family"] = "Times New Roman"
fig1 = plt.figure(figsize=(12,4) , dpi = 500)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , linear_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(linear_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("Linear Regression Train Dataset" , pad = 15 , fontsize = 15)
plt.text(0.4,2, "$R^2$ = {:.4f}".format(linear_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))
ax2 = plt.subplot(122)
plt.scatter(y_test , linear_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(linear_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("Linear Regression Test Dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2 , "$R^2$ = {:.4f}".format(linear_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))
for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3)  
fig1.savefig(figure_path + "Linear Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


<ipython-input-115-6779855c3106>:36: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig1.savefig(figure_path + "Linear Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-115-6779855c3106>:36: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig1.savefig(figure_path + "Linear Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [18]:
lr_MSE_train = mean_squared_error(y_train,linear_regression_train_Prediction)
lr_MAE_train = mean_absolute_error(y_train,linear_regression_train_Prediction)
lr_MSE_test  = mean_squared_error(y_test,linear_regression_test_Prediction)
lr_MAE_test  = mean_absolute_error(y_test,linear_regression_test_Prediction)

In [19]:
print(lr_MSE_train , lr_MAE_train , lr_MSE_test ,lr_MAE_test )

0.22491552865419545 0.33747139244305385 0.14082535181137865 0.27163329847340756


## 2. SVR MODEL

In [20]:
svr_regression_test_Prediction = svr_regression.predict(x_test)
svr_regression_train_Prediction = svr_regression.predict(x_train)


In [21]:
svr_regression_trainR2 = svr_regression.score(x_train,y_train)
svr_regression_testR2 = svr_regression.score(x_test,y_test)
print(svr_regression_trainR2 , svr_regression_testR2)

0.9400412235053256 0.6612545608838767


In [22]:
result_svr_train = pd.DataFrame()
result_svr_train["Viscosity (train)"] = y_train
result_svr_train["Predicted Viscosity (train)"] = svr_regression_train_Prediction
result_svr_train["Residues"] = abs(result_svr_train["Viscosity (train)"] - result_svr_train["Predicted Viscosity (train)"])
result_svr_train["Absolute Error (%)"] = result_svr_train["Residues"]*100/result_svr_train["Viscosity (train)"]

result_svr_test = pd.DataFrame()
result_svr_test["Viscosity (test)"] = y_test
result_svr_test["Predicted Viscosity (test)"] = svr_regression_test_Prediction
result_svr_test["Residues"] = abs(result_svr_test["Viscosity (test)"] - result_svr_test["Predicted Viscosity (test)"])
result_svr_test["Absolute Error (%)"] = result_svr_test["Residues"]*100/result_svr_test["Viscosity (test)"]




In [23]:
result_svr_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.903895,0.104853,18.715346
std,0.714770,0.668771,0.139937,31.657374
min,0.033511,0.133531,0.000400,0.023312
25%,0.403509,0.401574,0.052025,5.980388
50%,0.625668,0.669764,0.099645,11.300899
75%,1.250000,1.295298,0.100284,21.484146
max,3.010000,2.922394,1.486181,298.468745


In [24]:
result_svr_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.811157,0.201849,40.048473
std,0.593976,0.583122,0.279793,71.451550
min,0.027323,-0.296043,0.000576,0.105670
25%,0.367331,0.353623,0.048374,9.254976
50%,0.592982,0.662378,0.105720,18.907753
75%,1.106952,1.176426,0.223856,36.031739
max,2.435695,2.518486,1.786043,388.107866


In [116]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"
fig_svr = plt.figure(figsize=(12,4) , dpi = 500)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , svr_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(svr_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("SVR regression train dataset" , pad = 15 , fontsize = 15)
plt.text(0.4,2 , "$R^2$ = {:.4f}".format(svr_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

ax2 = plt.subplot(122)
plt.scatter(y_test , svr_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(svr_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("SVR regression test dataset" , pad = 15 , fontsize = 15)
plt.text(0.4,2 , "$R^2$ = {:.4f}".format(svr_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))
for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3)  
fig_svr.savefig(figure_path + "Svr Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-116-12cabdc75942>:38: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_svr.savefig(figure_path + "Svr Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-116-12cabdc75942>:38: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_svr.savefig(figure_path + "Svr Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [26]:
svr_MSE_train = mean_squared_error(y_train,svr_regression_train_Prediction)
svr_MAE_train = mean_absolute_error(y_train,svr_regression_train_Prediction)
svr_MSE_test  = mean_squared_error(y_test,svr_regression_test_Prediction)
svr_MAE_test  = mean_absolute_error(y_test,svr_regression_test_Prediction)
print(svr_MSE_train , svr_MAE_train , svr_MSE_test ,  svr_MAE_test )

0.030477194677723928 0.10485278750562164 0.11810594548273448 0.20184878387105812


## 3. Decision Tree Model 

In [28]:
dt_regression_test_Prediction = dt_regression.predict(x_test)
dt_regression_train_Prediction = dt_regression.predict(x_train)


In [29]:
dt_regression_trainR2 = dt_regression.score(x_train,y_train)
dt_regression_testR2 = dt_regression.score(x_test,y_test)
print(dt_regression_trainR2 , dt_regression_testR2)

0.9763837347437081 0.2858721184026072


In [30]:
result_dt_train = pd.DataFrame()
result_dt_train["Viscosity (train)"] = y_train
result_dt_train["Predicted Viscosity (train)"] = dt_regression_train_Prediction
result_dt_train["Residues"] = abs(result_dt_train["Viscosity (train)"] - result_dt_train["Predicted Viscosity (train)"])
result_dt_train["Absolute Error (%)"] = result_dt_train["Residues"]*100/result_dt_train["Viscosity (train)"]

result_dt_test = pd.DataFrame()
result_dt_test["Viscosity (test)"] = y_test
result_dt_test["Predicted Viscosity (test)"] = dt_regression_test_Prediction
result_dt_test["Residues"] = abs(result_dt_test["Viscosity (test)"] - result_dt_test["Predicted Viscosity (test)"])
result_dt_test["Absolute Error (%)"] = result_dt_test["Residues"]*100/result_dt_test["Viscosity (test)"]




In [31]:
result_dt_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.905874,0.053948,6.224217
std,0.714770,0.706279,0.095605,10.091718
min,0.033511,0.040411,0.000000,0.000000
25%,0.403509,0.402500,0.002212,0.472799
50%,0.625668,0.646745,0.016355,3.127355
75%,1.250000,1.227534,0.066667,7.326619
max,3.010000,2.934816,0.593333,77.564149


In [32]:
result_dt_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.768545,0.269899,29.644523
std,0.593976,0.642423,0.422181,40.780973
min,0.027323,0.040411,0.003294,0.823488
25%,0.367331,0.392534,0.027005,6.018458
50%,0.592982,0.515000,0.079000,17.045455
75%,1.106952,1.000000,0.288770,39.047619
max,2.435695,2.620000,1.870000,249.333333


In [117]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"
fig_dt = plt.figure(figsize=(12,4) , dpi = 500)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , dt_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(dt_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("Decision tree regression train dataset" , pad = 15 , fontsize = 15)
plt.text(0.4,2 , "$R^2$ = {:.4f}".format(dt_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

ax2 = plt.subplot(122)
plt.scatter(y_test , dt_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(dt_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("Decision tree regression test dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2 , "$R^2$ = {:.4f}".format(dt_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3)  
fig_dt.savefig(figure_path + "Dt Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-117-e26c3610b5cb>:39: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_dt.savefig(figure_path + "Dt Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-117-e26c3610b5cb>:39: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_dt.savefig(figure_path + "Dt Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [34]:
dt_MSE_train = mean_squared_error(y_train,dt_regression_train_Prediction)
dt_MAE_train = mean_absolute_error(y_train,dt_regression_train_Prediction)
dt_MSE_test  = mean_squared_error(y_test,dt_regression_test_Prediction)
dt_MAE_test  = mean_absolute_error(y_test,dt_regression_test_Prediction)
print(dt_MSE_train , dt_MAE_train , dt_MSE_test ,  dt_MAE_test )

0.012004206153884938 0.0539476040609137 0.24898563615119054 0.26989899803921574


## 4. Random forest Model 

In [35]:
rf_regression_test_Prediction = rf_regression.predict(x_test)
rf_regression_train_Prediction = rf_regression.predict(x_train)


In [36]:
rf_regression_trainR2 = rf_regression.score(x_train,y_train)
rf_regression_testR2 = rf_regression.score(x_test,y_test)
print(rf_regression_trainR2 , rf_regression_testR2)

0.9804097642678988 0.8147120839068794


In [37]:
result_rf_train = pd.DataFrame()
result_rf_train["Viscosity (train)"] = y_train
result_rf_train["Predicted Viscosity (train)"] = rf_regression_train_Prediction
result_rf_train["Residues"] = abs(result_rf_train["Viscosity (train)"] - result_rf_train["Predicted Viscosity (train)"])
result_rf_train["Absolute Error (%)"] = result_rf_train["Residues"]*100/result_rf_train["Viscosity (train)"]

result_rf_test = pd.DataFrame()
result_rf_test["Viscosity (test)"] = y_test
result_rf_test["Predicted Viscosity (test)"] = rf_regression_test_Prediction
result_rf_test["Residues"] = abs(result_rf_test["Viscosity (test)"] - result_rf_test["Predicted Viscosity (test)"])
result_rf_test["Absolute Error (%)"] = result_rf_test["Residues"]*100/result_rf_test["Viscosity (test)"]




In [38]:
result_rf_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.899698,0.058770,10.419230
std,0.714770,0.665372,0.080852,24.855680
min,0.033511,0.084666,0.000005,0.001188
25%,0.403509,0.412840,0.010058,1.758706
50%,0.625668,0.640795,0.030570,4.215092
75%,1.250000,1.261618,0.073870,9.337520
max,3.010000,2.891186,0.617955,192.458118


In [39]:
result_rf_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.813931,0.154997,27.736264
std,0.593976,0.537063,0.202635,43.525210
min,0.027323,0.084666,0.000313,0.014148
25%,0.367331,0.409608,0.029825,6.367447
50%,0.592982,0.643215,0.059344,14.227240
75%,1.106952,1.170413,0.202794,25.898862
max,2.435695,2.506992,1.082787,212.311082


In [118]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"
fig_rf = plt.figure(figsize=(12,4) , dpi = 500)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , rf_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(rf_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("Random forest regression train dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2 , "$R^2$ = {:.4f}".format(rf_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))


ax2 = plt.subplot(122)
plt.scatter(y_test , rf_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(rf_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("Random forest regression test dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2  , "$R^2$ = {:.4f}".format(rf_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3)  
fig_rf.savefig(figure_path + "RF Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-118-5cc89879cd66>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig_rf = plt.figure(figsize=(12,4) , dpi = 500)
<ipython-input-118-5cc89879cd66>:40: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_rf.savefig(figure_path + "RF Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-118-5cc89879cd66>:40: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  

In [41]:
rf_MSE_train = mean_squared_error(y_train,rf_regression_train_Prediction)
rf_MAE_train = mean_absolute_error(y_train,rf_regression_train_Prediction)
rf_MSE_test  = mean_squared_error(y_test,rf_regression_test_Prediction)
rf_MAE_test  = mean_absolute_error(y_test,rf_regression_test_Prediction)
print(rf_MSE_train , rf_MAE_train , rf_MSE_test , rf_MAE_test )

0.009957765369725046 0.05877002503955434 0.06460191633517993 0.15499717430481286


## 5. KNN Model

In [42]:
knn_regression_test_Prediction = knn_regression.predict(x_test)
knn_regression_train_Prediction =knn_regression.predict(x_train)


In [43]:
knn_regression_trainR2 = knn_regression.score(x_train,y_train)
knn_regression_testR2 = knn_regression.score(x_test,y_test)
print(knn_regression_trainR2 , knn_regression_testR2)

0.909253222194798 0.6632257283678096


In [44]:
result_knn_train = pd.DataFrame()
result_knn_train["Viscosity (train)"] = y_train
result_knn_train["Predicted Viscosity (train)"] = knn_regression_train_Prediction
result_knn_train["Residues"] = abs(result_knn_train["Viscosity (train)"] - result_knn_train["Predicted Viscosity (train)"])
result_knn_train["Absolute Error (%)"] = result_knn_train["Residues"]*100/result_knn_train["Viscosity (train)"]

result_knn_test = pd.DataFrame()
result_knn_test["Viscosity (test)"] = y_test
result_knn_test["Predicted Viscosity (test)"] = knn_regression_test_Prediction
result_knn_test["Residues"] = abs(result_knn_test["Viscosity (test)"] - result_knn_test["Predicted Viscosity (test)"])
result_knn_test["Absolute Error (%)"] = result_knn_test["Residues"]*100/result_knn_test["Viscosity (test)"]




In [45]:
result_knn_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.899538,0.120018,13.897286
std,0.714770,0.670141,0.178562,22.124239
min,0.033511,0.044189,0.000000,0.000000
25%,0.403509,0.419945,0.010911,2.272727
50%,0.625668,0.646745,0.044543,6.648459
75%,1.250000,1.227534,0.140851,16.889483
max,3.010000,2.944198,0.856667,178.181818


In [46]:
result_knn_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.783535,0.195002,22.277499
std,0.593976,0.601634,0.283440,26.839441
min,0.027323,0.044189,0.002088,1.690829
25%,0.367331,0.366247,0.027693,5.012882
50%,0.592982,0.574866,0.058318,11.975120
75%,1.106952,1.088235,0.232621,34.935911
max,2.435695,2.515267,1.184492,196.732026


In [119]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"


fig_knn = plt.figure(figsize=(12,4) , dpi = 500)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , knn_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(knn_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("KNN regression train dataset" , pad = 15 , fontsize = 15)
plt.text(0.4,2  , "$R^2$ = {:.4f}".format(knn_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))


ax2 = plt.subplot(122)
plt.scatter(y_test , knn_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(knn_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("KNN regression test dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2  , "$R^2$ = {:.4f}".format(knn_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3)   

fig_knn.savefig(figure_path + "KNN Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-119-7ed636127598>:43: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_knn.savefig(figure_path + "KNN Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-119-7ed636127598>:43: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_knn.savefig(figure_path + "KNN Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [48]:
knn_MSE_train = mean_squared_error(y_train,knn_regression_train_Prediction)
knn_MAE_train = mean_absolute_error(y_train,knn_regression_train_Prediction)
knn_MSE_test  = mean_squared_error(y_test,knn_regression_test_Prediction)
knn_MAE_test  = mean_absolute_error(y_test,knn_regression_test_Prediction)
print(knn_MSE_train , knn_MAE_train , knn_MSE_test , knn_MAE_test )

0.04612681204045177 0.12001757191201355 0.11741868427561046 0.1950018117647059


## 6. ANN Model

In [49]:
ann_regression_test_Prediction = ann_regression.predict(x_test)
ann_regression_train_Prediction = ann_regression.predict(x_train)


In [50]:
from sklearn.metrics import r2_score
ann_regression_trainR2 = r2_score(y_train , ann_regression_train_Prediction)
ann_regression_testR2 = r2_score(y_test , ann_regression_test_Prediction)
print(ann_regression_trainR2 , ann_regression_testR2)

0.9443899889205869 0.6775760121272241


In [51]:
result_ann_train = pd.DataFrame()
result_ann_train["Viscosity (train)"] = y_train
result_ann_train["Predicted Viscosity (train)"] = ann_regression_train_Prediction
result_ann_train["Residues"] = abs(result_ann_train["Viscosity (train)"] - result_ann_train["Predicted Viscosity (train)"])
result_ann_train["Absolute Error (%)"] = result_ann_train["Residues"]*100/result_ann_train["Viscosity (train)"]

result_ann_test = pd.DataFrame()
result_ann_test["Viscosity (test)"] = y_test
result_ann_test["Predicted Viscosity (test)"] = ann_regression_test_Prediction
result_ann_test["Residues"] = abs(result_ann_test["Viscosity (test)"] - result_ann_test["Predicted Viscosity (test)"])
result_ann_test["Absolute Error (%)"] = result_ann_test["Residues"]*100/result_ann_test["Viscosity (test)"]




In [52]:
result_ann_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.937119,0.083772,11.832649
std,0.714770,0.742065,0.146142,22.709316
min,0.033511,0.061821,0.000300,0.035307
25%,0.403509,0.411154,0.021321,2.845069
50%,0.625668,0.627641,0.040812,6.320665
75%,1.250000,1.365487,0.089264,12.640696
max,3.010000,3.104177,1.230574,264.847408


In [53]:
result_ann_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.826815,0.198289,26.170165
std,0.593976,0.614561,0.271969,26.734417
min,0.027323,0.061493,0.000927,0.148544
25%,0.367331,0.384129,0.034142,6.717141
50%,0.592982,0.617420,0.073475,19.946464
75%,1.106952,1.151002,0.238155,34.529207
max,2.435695,2.816707,1.219301,125.982111


In [120]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"


fig_ann = plt.figure(figsize=(12,4) , dpi = 500)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , ann_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(ann_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("ANN regression train dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2 , "$R^2$ = {:.4f}".format(ann_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))


ax2 = plt.subplot(122)
plt.scatter(y_test , ann_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(ann_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("ANN regression test dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2  , "$R^2$ = {:.4f}".format(ann_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3) 

fig_ann.savefig(figure_path + "ANN Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-120-fd9a93efd845>:43: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_ann.savefig(figure_path + "ANN Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-120-fd9a93efd845>:43: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_ann.savefig(figure_path + "ANN Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [55]:
ann_MSE_train = mean_squared_error(y_train,ann_regression_train_Prediction)
ann_MAE_train = mean_absolute_error(y_train,ann_regression_train_Prediction)
ann_MSE_test  = mean_squared_error(y_test,ann_regression_test_Prediction)
ann_MAE_test  = mean_absolute_error(y_test,ann_regression_test_Prediction)
print(ann_MSE_train , ann_MAE_train , ann_MSE_test , ann_MAE_test )

0.02826670644035235 0.0837717868768576 0.11241535836877758 0.19828905649865652


## 7. XGB Model

In [56]:
xgb_regression_test_Prediction = xgb_regression.predict(x_test)
xgb_regression_train_Prediction = xgb_regression.predict(x_train)


In [57]:
from sklearn.metrics import r2_score
xgb_regression_trainR2 = r2_score(y_train , xgb_regression_train_Prediction)
xgb_regression_testR2 = r2_score(y_test , xgb_regression_test_Prediction)
print(xgb_regression_trainR2 , xgb_regression_testR2)

0.9672143661239633 0.7023470244466465


In [58]:
result_xgb_train = pd.DataFrame()
result_xgb_train["Viscosity (train)"] = y_train
result_xgb_train["Predicted Viscosity (train)"] = xgb_regression_train_Prediction
result_xgb_train["Residues"] = abs(result_xgb_train["Viscosity (train)"] - result_xgb_train["Predicted Viscosity (train)"])
result_xgb_train["Absolute Error (%)"] = result_xgb_train["Residues"]*100/result_xgb_train["Viscosity (train)"]

result_xgb_test = pd.DataFrame()
result_xgb_test["Viscosity (test)"] = y_test
result_xgb_test["Predicted Viscosity (test)"] = xgb_regression_test_Prediction
result_xgb_test["Residues"] = abs(result_xgb_test["Viscosity (test)"] - result_xgb_test["Predicted Viscosity (test)"])
result_xgb_test["Absolute Error (%)"] = result_xgb_test["Residues"]*100/result_xgb_test["Viscosity (test)"]




In [121]:
#Make all rc parameter as default 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"


fig_xgb= plt.figure(figsize=(12,4) , dpi = 500)

plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

mpl.rcParams['xtick.major.size'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.size'] = 2
mpl.rcParams['ytick.major.width'] = 2

ax1 = plt.subplot(121)
plt.scatter(y_train , xgb_regression_train_Prediction , c = 'blue'  , s = 10 , label = "$R^2$ = {:.4f}".format(xgb_regression_trainR2))
plt.plot(y_train,y_train, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("XG boost regression train dataset" , pad = 15 , fontsize = 15)
plt.text( 0.4,2  , "$R^2$ = {:.4f}".format(xgb_regression_trainR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))


ax2 = plt.subplot(122)
plt.scatter(y_test , xgb_regression_test_Prediction , c = 'green'  , s = 10 , label = "$R^2$ = {:.4f}".format(xgb_regression_testR2))
plt.plot(y_test,y_test, linewidth = 3 , c = 'red')
plt.xlabel("Laboratory Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.ylabel("Predicted Viscosity (cp)", fontsize = 15 , labelpad = 10)
plt.title("XG boost regression test dataset" , pad = 15 , fontsize = 15)
plt.text(0.4,2 , "$R^2$ = {:.4f}".format(xgb_regression_testR2) , fontsize = 14)
plt.xlim((0,3))
plt.ylim((0,3))

for i in range(1,3,1):
    for axis in ['top', 'bottom', 'left', 'right']:
        locals()["ax"+str(i)].spines[axis].set_linewidth(2)
        
plt.subplots_adjust(wspace=0.3,hspace=0.3)  

fig_xgb.savefig(figure_path + "XGB Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-121-0d7832d02f10>:43: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_xgb.savefig(figure_path + "XGB Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-121-0d7832d02f10>:43: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_xgb.savefig(figure_path + "XGB Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [60]:
xgb_MSE_train = mean_squared_error(y_train,xgb_regression_train_Prediction)
xgb_MAE_train = mean_absolute_error(y_train,xgb_regression_train_Prediction)
xgb_MSE_test  = mean_squared_error(y_test,xgb_regression_test_Prediction)
xgb_MAE_test  = mean_absolute_error(y_test,xgb_regression_test_Prediction)
print(xgb_MSE_train , xgb_MAE_train , xgb_MSE_test , xgb_MAE_test )

0.016665018946163866 0.08429180738105872 0.10377877321449909 0.18565987760760364


In [61]:
result_xgb_train.describe()

,Viscosity (train),Predicted Viscosity (train),Residues,Absolute Error (%)
count,197.000000,197.000000,197.000000,197.000000
mean,0.905874,0.905811,0.084292,15.734383
std,0.714770,0.671309,0.098024,37.074907
min,0.033511,0.153815,0.001418,0.277029
25%,0.403509,0.418480,0.024773,3.677912
50%,0.625668,0.621519,0.052757,7.451444
75%,1.250000,1.341959,0.110737,15.192391
max,3.010000,2.790652,0.845398,358.999209


In [62]:
result_xgb_test.describe()

,Viscosity (test),Predicted Viscosity (test),Residues,Absolute Error (%)
count,85.000000,85.000000,85.000000,85.000000
mean,0.799146,0.822603,0.185660,40.062370
std,0.593976,0.570203,0.264829,84.301063
min,0.027323,0.153815,0.000177,0.041466
25%,0.367331,0.401859,0.036303,5.743357
50%,0.592982,0.585263,0.083125,17.334730
75%,1.106952,1.159266,0.209701,31.359825
max,2.435695,2.432097,1.555751,462.951450


In [63]:
model_comp_test = pd.DataFrame({
    'Model Name' : ["Linear Regression" , 'SVM' , 'Decision Tree' , 'Random Forest' , 'KNN' , 'ANN',"XGB"],
    'R Square' : [linear_regression_testR2  , svr_regression_testR2 ,dt_regression_testR2,rf_regression_testR2,knn_regression_testR2,ann_regression_testR2,xgb_regression_testR2],
    'MSE' : [lr_MSE_test  , svr_MSE_test , dt_MSE_test ,  rf_MSE_test ,knn_MSE_test,ann_MSE_test,xgb_MSE_test],
    'MAE' : [lr_MAE_test  , svr_MAE_test , dt_MAE_test ,  rf_MAE_test ,knn_MAE_test,ann_MAE_test,xgb_MAE_test],
    '%AAD' : [result_lr_test.describe()["Absolute Error (%)"].loc["mean"],
              result_svr_test.describe()["Absolute Error (%)"].loc["mean"],
              result_dt_test.describe()["Absolute Error (%)"].loc["mean"],
              result_rf_test.describe()["Absolute Error (%)"].loc["mean"],
              result_knn_test.describe()["Absolute Error (%)"].loc["mean"],
              result_ann_test.describe()["Absolute Error (%)"].loc["mean"],
              result_xgb_test.describe()["Absolute Error (%)"].loc["mean"]
             ]

})

In [64]:
model_comp_test

,Model Name,R Square,MSE,MAE,%AAD
0,Linear Regression,0.596092,0.140825,0.271633,50.803625
1,SVM,0.661255,0.118106,0.201849,40.048473
2,Decision Tree,0.285872,0.248986,0.269899,29.644523
3,Random Forest,0.814712,0.064602,0.154997,27.736264
4,KNN,0.663226,0.117419,0.195002,22.277499
5,ANN,0.677576,0.112415,0.198289,26.170165
6,XGB,0.702347,0.103779,0.185660,40.062370


In [65]:
model_comp_train = pd.DataFrame({
    'Model Name' : ["Linear Regression"  , 'SVM' , 'Decision Tree' , 'Random Forest' , 'KNN','ANN',"XGB"],
    'R Square' : [linear_regression_trainR2  , svr_regression_trainR2 ,dt_regression_trainR2,rf_regression_trainR2,knn_regression_trainR2,ann_regression_trainR2,xgb_regression_trainR2],
    'MSE' : [lr_MSE_train , svr_MSE_train , dt_MSE_train ,  rf_MSE_train ,knn_MSE_train,ann_MSE_train,xgb_MSE_train],
    'MAE' : [lr_MAE_train  , svr_MAE_train , dt_MAE_train ,  rf_MAE_train ,knn_MAE_train,ann_MAE_train,xgb_MSE_train],
    '%AAD' : [result_lr_train.describe()["Absolute Error (%)"].loc["mean"],
              result_svr_train.describe()["Absolute Error (%)"].loc["mean"],
              result_dt_train.describe()["Absolute Error (%)"].loc["mean"],
              result_rf_train.describe()["Absolute Error (%)"].loc["mean"],
              result_knn_train.describe()["Absolute Error (%)"].loc["mean"],
              result_ann_train.describe()["Absolute Error (%)"].loc["mean"],
             result_xgb_train.describe()["Absolute Error (%)"].loc["mean"]]
})

In [66]:
model_comp_train

,Model Name,R Square,MSE,MAE,%AAD
0,Linear Regression,0.557516,0.224916,0.337471,57.085121
1,SVM,0.940041,0.030477,0.104853,18.715346
2,Decision Tree,0.976384,0.012004,0.053948,6.224217
3,Random Forest,0.980410,0.009958,0.058770,10.419230
4,KNN,0.909253,0.046127,0.120018,13.897286
5,ANN,0.944390,0.028267,0.083772,11.832649
6,XGB,0.967214,0.016665,0.016665,15.734383


## Preparing Dataframe containing result of all models evaluation parameters

#### 1. For test dataset

In [67]:


model_comp_test_paper = pd.DataFrame({
    'Model Name' : ['LR','SVR' , 'DT' , 'RF' , 'KNN' , 'ANN',"XGB"],
    'R Square' : [linear_regression_testR2, svr_regression_testR2 ,dt_regression_testR2,rf_regression_testR2,knn_regression_testR2,ann_regression_testR2,xgb_regression_testR2],
    'MSE' : [lr_MSE_test,svr_MSE_test , dt_MSE_test ,  rf_MSE_test ,knn_MSE_test,ann_MSE_test,xgb_MSE_test],
    'MAE' : [lr_MAE_test,svr_MAE_test , dt_MAE_test ,  rf_MAE_test ,knn_MAE_test,ann_MAE_test,xgb_MAE_test],
    '%AARD' : [result_lr_test.describe()["Absolute Error (%)"].loc["mean"],
              result_svr_test.describe()["Absolute Error (%)"].loc["mean"],
              result_dt_test.describe()["Absolute Error (%)"].loc["mean"],
              result_rf_test.describe()["Absolute Error (%)"].loc["mean"],
              result_knn_test.describe()["Absolute Error (%)"].loc["mean"],
              result_ann_test.describe()["Absolute Error (%)"].loc["mean"],
              result_xgb_test.describe()["Absolute Error (%)"].loc["mean"]
             ]

})

In [68]:
model_comp_test_paper.to_excel(export_data_path+"Result_Compare_test.xlsx",index = False)

#### 2. For train dataset

In [69]:
model_comp_train_paper = pd.DataFrame({
    'Model Name' : ['LR','SVR' , 'DT' , 'RF' , 'KNN' , 'ANN',"XGB"],
    'R Square' : [linear_regression_trainR2,svr_regression_trainR2 ,dt_regression_trainR2,rf_regression_trainR2,knn_regression_trainR2,ann_regression_trainR2,xgb_regression_trainR2],
    'MSE' : [lr_MSE_train,svr_MSE_train , dt_MSE_train ,  rf_MSE_train ,knn_MSE_train,ann_MSE_train,xgb_MSE_train],
    'MAE' : [ lr_MAE_train , svr_MAE_train , dt_MAE_train ,  rf_MAE_train ,knn_MAE_train,ann_MAE_train,xgb_MSE_train],
    '%AARD' : [result_lr_train.describe()["Absolute Error (%)"].loc["mean"],
              result_svr_train.describe()["Absolute Error (%)"].loc["mean"],
              result_dt_train.describe()["Absolute Error (%)"].loc["mean"],
              result_rf_train.describe()["Absolute Error (%)"].loc["mean"],
              result_knn_train.describe()["Absolute Error (%)"].loc["mean"],
              result_ann_train.describe()["Absolute Error (%)"].loc["mean"],
             result_xgb_train.describe()["Absolute Error (%)"].loc["mean"]]
})

In [70]:
model_comp_train_paper.to_excel(export_data_path+"Result_Compare_train.xlsx")

In [71]:
#Creating Dataframe For bar plot by adding label train and test and merging both dataframe

model_comp_test_paper["Label"] = "Test"
model_comp_train_paper["Label"] = "Train"
Final_bar_plot = model_comp_test_paper.append(model_comp_train_paper)
Final_bar_plot

,Model Name,R Square,MSE,MAE,%AARD,Label
0,LR,0.596092,0.140825,0.271633,50.803625,Test
1,SVR,0.661255,0.118106,0.201849,40.048473,Test
2,DT,0.285872,0.248986,0.269899,29.644523,Test
3,RF,0.814712,0.064602,0.154997,27.736264,Test
4,KNN,0.663226,0.117419,0.195002,22.277499,Test
5,ANN,0.677576,0.112415,0.198289,26.170165,Test
6,XGB,0.702347,0.103779,0.185660,40.062370,Test
0,LR,0.557516,0.224916,0.337471,57.085121,Train
1,SVR,0.940041,0.030477,0.104853,18.715346,Train
2,DT,0.976384,0.012004,0.053948,6.224217,Train


## Bar plot which comparing the % AARD values

In [101]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"
plt.rc('xtick', labelsize=17)
plt.rc('ytick', labelsize=17)


mpl.rcParams['xtick.major.size'] = 3
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 3
mpl.rcParams['ytick.major.width'] = 3
fig_aard = plt.figure(figsize = (15,5) , dpi = 200)

ax = sns.barplot( x = "Model Name" , y = "%AARD" , data = Final_bar_plot , hue = "Label" , saturation = 10)

plt.legend(fontsize = 15 ,  frameon=False )
plt.ylabel("% AARD" , fontsize = 15 , labelpad = 12)
plt.ylim((0,60))
ax.set(xlabel = None)


ax = fig_aard.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.9)

fig_aard.savefig(figure_path + "AARD Result.tif" ,quality = 95 , dpi = 500 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-101-6761bd9df3f9>:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "quality" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_aard.savefig(figure_path + "AARD Result.tif" ,quality = 95 , dpi = 500 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-101-6761bd9df3f9>:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_aard.savefig(figure_path + "AARD Result.tif" ,quality = 95 , dpi = 500 ,papertype = "letter",bbox_inches = "tight")


In [103]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "Times New Roman"
plt.rc('xtick', labelsize=17)
plt.rc('ytick', labelsize=17)


mpl.rcParams['xtick.major.size'] = 3
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 3
mpl.rcParams['ytick.major.width'] = 3
fig_mse = plt.figure(figsize = (12,5) , dpi = 200)

ax = sns.barplot( x = "Model Name" , y = "MSE" , data = Final_bar_plot , hue = "Label" , saturation = 10)

plt.legend(fontsize = 15 ,  frameon=False )
plt.ylabel("MSE" , fontsize = 15 , labelpad = 12)
ax.set(xlabel = None)
plt.ylim((0,0.30))


ax = fig_mse.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.9)

fig_mse.savefig(figure_path + "MSE Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-103-565878687a72>:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_mse.savefig(figure_path + "MSE Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-103-565878687a72>:25: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_mse.savefig(figure_path + "MSE Result.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


## Creating Dataframes for closeness result plots and comaparision for residuals plots

In [74]:
lr_plot_result = result_lr_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)
svr_plot_result = result_svr_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)
dt_plot_result = result_dt_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)
rf_plot_result = result_rf_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)
knn_plot_result = result_knn_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)
ann_plot_result = result_ann_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)
xgb_plot_result = result_xgb_test[["Viscosity (test)","Predicted Viscosity (test)"]].reset_index(drop=True)

lr_plot_result["Residual"] = lr_plot_result["Viscosity (test)"] - lr_plot_result["Predicted Viscosity (test)"]
svr_plot_result["Residual"] = svr_plot_result["Viscosity (test)"] - svr_plot_result["Predicted Viscosity (test)"]
dt_plot_result["Residual"] = dt_plot_result["Viscosity (test)"] - dt_plot_result["Predicted Viscosity (test)"]
rf_plot_result["Residual"] = rf_plot_result["Viscosity (test)"] - rf_plot_result["Predicted Viscosity (test)"]
knn_plot_result["Residual"] = knn_plot_result["Viscosity (test)"] - knn_plot_result["Predicted Viscosity (test)"]
ann_plot_result["Residual"] = ann_plot_result["Viscosity (test)"] - ann_plot_result["Predicted Viscosity (test)"]
xgb_plot_result["Residual"] = xgb_plot_result["Viscosity (test)"] - xgb_plot_result["Predicted Viscosity (test)"]

## scatter plot which comparing the residuals values for entire dataset for each saturation pressure records


In [75]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_compare_res = plt.figure(figsize = (25,20) , dpi = 200)
plt.scatter(lr_plot_result["Viscosity (test)"] ,lr_plot_result["Residual"] , marker = "P" , s = 200 , c = "Cyan",label = "LR",alpha = 0.7)
plt.scatter(svr_plot_result["Viscosity (test)"] ,svr_plot_result["Residual"] , marker = "^" , s = 200 , c = "blue",label = "SVR",alpha = 0.7)
plt.scatter(dt_plot_result["Viscosity (test)"] ,dt_plot_result["Residual"] , marker = "D" , s = 200 , c = "red",label = "DT",alpha = 0.7)
plt.scatter(rf_plot_result["Viscosity (test)"] ,rf_plot_result["Residual"] , marker = "8" , s = 200 , c = "green",label = "RF",alpha = 0.7)
plt.scatter(knn_plot_result["Viscosity (test)"] ,knn_plot_result["Residual"] , marker = "o" , s = 200 , c = "purple",label = "KNN",alpha = 0.7)
plt.scatter(ann_plot_result["Viscosity (test)"] ,ann_plot_result["Residual"] , marker = "s" , s = 200 , c = "maroon",label = "ANN",alpha = 0.7)
plt.scatter(xgb_plot_result["Viscosity (test)"] ,xgb_plot_result["Residual"] , marker = "*" , s = 200 , c = "goldenrod",label = "XGB",alpha = 0.7)


plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Laboratory Viscosity (cp)" , fontsize = 35, labelpad = 20)
plt.ylabel("Residuals (cp)",fontsize = 35, labelpad = 20)
plt.xlim((0,None))

#Defining axis of graph and increasing width of boarder of box
ax = fig_compare_res.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)
    
plt.axhline(y = 0, color = 'black',ls = "-" ,lw = 3)


fig_compare_res.savefig(figure_path + "fig_compare_res.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-75-b02a5da094cc>:33: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_compare_res.savefig(figure_path + "fig_compare_res.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-75-b02a5da094cc>:33: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_compare_res.savefig(figure_path + "fig_compare_res.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [104]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_linear2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(lr_plot_result.index ,lr_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(svr_plot_result.index ,svr_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(lr_plot_result.index ,lr_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "LR Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)
plt.xlim(None,85)
plt.ylim(None,3)

#Defining axis of graph and increasing width of boarder of box
ax = fig_linear2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)


fig_linear2.savefig(figure_path + "linear_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-104-2e0c7a21662c>:28: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_linear2.savefig(figure_path + "linear_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-104-2e0c7a21662c>:28: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_linear2.savefig(figure_path + "linear_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [105]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_svr2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(svr_plot_result.index ,svr_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(svr_plot_result.index ,svr_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(svr_plot_result.index ,svr_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "SVR Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)
plt.xlim(None,85)
plt.ylim(None,3)
#Defining axis of graph and increasing width of boarder of box
ax = fig_svr2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)


fig_svr2.savefig(figure_path + "svr_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-105-149f5d74847b>:27: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_svr2.savefig(figure_path + "svr_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-105-149f5d74847b>:27: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_svr2.savefig(figure_path + "svr_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [106]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_dt2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(dt_plot_result.index ,dt_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(dt_plot_result.index ,dt_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(dt_plot_result.index ,dt_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "DT Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)
plt.xlim(None,85)
plt.ylim(None,3)
#Defining axis of graph and increasing width of boarder of box
ax = fig_dt2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)


fig_dt2.savefig(figure_path + "dt_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-106-cd3e93ac2de4>:27: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_dt2.savefig(figure_path + "dt_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-106-cd3e93ac2de4>:27: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_dt2.savefig(figure_path + "dt_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [107]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_rf2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(rf_plot_result.index ,rf_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(rf_plot_result.index ,rf_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(rf_plot_result.index ,rf_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "RF Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)

plt.xlim(None,85)
plt.ylim(None,3)
#Defining axis of graph and increasing width of boarder of box
ax = fig_rf2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)


fig_rf2.savefig(figure_path + "rf_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")

<ipython-input-107-5b6984e51f73>:28: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_rf2.savefig(figure_path + "rf_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-107-5b6984e51f73>:28: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_rf2.savefig(figure_path + "rf_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [108]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_knn2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(knn_plot_result.index ,knn_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(knn_plot_result.index ,knn_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(knn_plot_result.index ,knn_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "KNN Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)

plt.xlim(None,85)
plt.ylim(None,3)

#Defining axis of graph and increasing width of boarder of box
ax = fig_knn2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)

fig_knn2.savefig(figure_path + "knn_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


<ipython-input-108-b7a026763b7b>:28: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_knn2.savefig(figure_path + "knn_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-108-b7a026763b7b>:28: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_knn2.savefig(figure_path + "knn_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [109]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_ann2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(ann_plot_result.index ,ann_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(ann_plot_result.index ,ann_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(ann_plot_result.index ,ann_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "ANN Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)

plt.xlim(None,85)
plt.ylim(None,3)

#Defining axis of graph and increasing width of boarder of box
ax = fig_ann2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)


fig_ann2.savefig(figure_path + "ann_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


<ipython-input-109-8f79b0da461b>:29: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_ann2.savefig(figure_path + "ann_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-109-8f79b0da461b>:29: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_ann2.savefig(figure_path + "ann_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")


In [110]:
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 6


fig_xgb2 = plt.figure(figsize = (30,15) , dpi = 200)
plt.scatter(xgb_plot_result.index ,xgb_plot_result[["Viscosity (test)"]] , marker = "^" , s = 250 , c = "blue",label = "Laboratory Value")
#plt.plot(xgb_plot_result.index ,xgb_plot_result[["Psat (test)"]] ,lw = 3 , c = "green" , label = "Laboratory Value")

plt.plot(xgb_plot_result.index ,xgb_plot_result[["Predicted Viscosity (test)"]] ,lw = 3 , c = "red",label = "XGB Predicted Value")

plt.legend(fontsize = 25,frameon=False)
plt.xlabel("Data Index" , fontsize = 35, labelpad = 20)
plt.ylabel("Viscosity (cp)",fontsize = 35, labelpad = 20)

plt.xlim(None,85)
plt.ylim(None,3)

#Defining axis of graph and increasing width of boarder of box
ax = fig_xgb2.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)

fig_xgb2.savefig(figure_path + "xgb_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")



<ipython-input-110-da8ff73f5c0f>:28: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  fig_xgb2.savefig(figure_path + "xgb_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
<ipython-input-110-da8ff73f5c0f>:28: MatplotlibDeprecationWarning: 
The 'quality' parameter of print_jpg() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. Use pil_kwargs={'quality': ...} instead. If any parameter follows 'quality', they should be passed as keyword, not positionally.
  fig_xgb2.savefig(figure_path + "xgb_line.jpg" ,quality = 95 , dpi = 300 ,papertype = "letter",bbox_inches = "tight")
